In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd

In [5]:
data = pd.read_excel("anksiyete_translated_to_tr.xlsx")

data = pd.read_excel("Data/depression_llm_filtered_en.xlsx")

In [6]:
pd.set_option('display.max_colwidth', None)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2632 entries, 0 to 2631
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Result           2632 non-null   object
 1   Text             2632 non-null   object
 2   Translated_Text  2623 non-null   object
dtypes: object(3)
memory usage: 61.8+ KB


data = data[data['Text'].apply(lambda x: isinstance(x, str))]

data.info()

In [8]:
data.head()

,Result,Text,Translated_Text
0,yes,My bf is back at work (in retail) and these customers coming in for unnecessary BS is really setting off my anxiety.,Bf'm işe geri döndü (perakende) ve gereksiz BS için gelen bu müşteriler gerçekten endişemi başlatıyor.
1,yes,I’ve been extremely anxious and have now fully lost my appetite and have not left my bed. I can’t stop imaging if I’ve developed lung cancer... experiencing health issues such as this is causing me to have panic episodes.,Son derece endişeliydim ve şimdi iştahımı tamamen kaybettim ve yatağımdan ayrılmadım. Akciğer kanseri geliştirdiysem görüntülemeyi durduramıyorum... böyle sağlık sorunları yaşamak panik atak geçirmeme neden oluyor.
2,yes,"I have a problem with super repetitive thoughts and worries. one of those is self-pity. self-pity leaks into everything i say, every mannerism i have, my whole personality and choices... it feels like my identity basically... my ego has never really developed beforehand and took a turn for the worse... I never developed a sense of self or could get myself to listen... I tend to outburst a lot... I associate my feelings with my entire identity because i have let them become my identity and i don't know how to undo that... I feel like i have just been raised to hate myself for some reason...","Süper tekrarlayan düşünceler ve endişeler ile ilgili bir sorunum var. Bunlardan biri kendine acıma. Kendi kendine acıma, söylediğim her şeye, sahip olduğum her tavıra, tüm kişiliğime ve seçimlerime sızıyor... Temelde kimliğim gibi geliyor... egom hiçbir zaman gerçekten önceden gelişmedi ve daha kötüsü için bir dönüş yapmadı... Ben asla bir benlik duygusu geliştirmedim ya da kendimi dinlemem için bir neden edinemedim... Duygularımı tüm kimliğimle ilişkilendirmeye meyilliyim..."
3,yes,"I have a history of anxiety attacks. I've been struggling from severe health anxiety that doesn't go away even after doing exams. I convinced myself I had HIV that was going to progress to AIDS... I thought I had a fatal illness... I started having bad shortness of breath and palpitations... I panicked nonetheless as I always think there's a possibility I'm a rare case... I felt like I had a clot in my arm and I needed to go to the ER now... I'm still struggling on it, the paranoia is still there.","Anksiyete atakları geçmişim var. Sınavlar yaptıktan sonra bile gitmeyen ciddi sağlık kaygılarından dolayı mücadele ediyorum. AIDS'e ilerleyecek HIV'im olduğuna kendimi ikna ettim. Ölümcül bir hastalığım olduğunu düşündüm... Nefes darlığı ve çarpıntılarım başladı... Sürekli panikledim, hala nadir bir vaka olduğumu düşündüğüm için panikledim... Kolumda bir pıhtı varmış gibi hissettim ve şimdi acil servise gitmek için ihtiyacım var."
4,no,I went in to my local coffee shop and ordered a mocha latte first time without being nervous. I feel really happy about it. Edit: Wow thank you for the awards. I appreciate it and nice comments,Yerel kahve dükkanıma gittim ve ilk kez sinirlenmeden bir mocha latte sipariş ettim. Bu konuda gerçekten mutluyum. Edit: Wow ödüller için teşekkür ederim. Bunu takdir ediyorum ve güzel yorumlar


In [6]:
data[data["is_depression"]==0]

,clean_text,is_depression
3831,switchfoot http twitpic com y zl awww that s a bummer you shoulda got david carr of third day to do it d,0
3832,is upset that he can t update his facebook by texting it and might cry a a result school today also blah,0
3833,kenichan i dived many time for the ball managed to save 0 the rest go out of bound,0
3834,my whole body feel itchy and like it on fire,0
3835,nationwideclass no it s not behaving at all i m mad why am i here because i can t see you all over there,0
...,...,...
7726,is that snow,0
7727,moulin rouge mad me cry once again,0
7728,trying to shout but can t find people on the list,0
7729,ughh can t find my red sox hat got ta wear this creepy nick pirro version,0


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def batch_translate_text(texts, batch_size=8):
    translated_texts = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translated_texts.extend(translated_batch)
    return translated_texts

texts = data['clean_text'].tolist()
translated_texts = batch_translate_text(texts)

data['Translated_Text'] = translated_texts

print(data)

C:\Users\ensar\Desktop\dogaldilisleme\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
data

In [ ]:
data.to_excel("depression_translated_to_tr.xlsx", index=False)